In [108]:
import experience_study as exp
import os
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from config import db, app

from models import Hive, ExplanatoryVariable
from datetime import datetime

import data_processing as dclean

In [109]:
explanatory_variables, hives = dclean.collect_data(app)
hives_mod = dclean.rename_ids(hives)
df_normalized = dclean.normalize_data(hives_mod)

df_normalized.head()

,feeding,capped_brood,fate,inspection_id,humidity,treatment,varroa_mites,egg_count,activity_surrounding_hive,super_count,...,species,date_introduced,status,origin,hive_id,date_added,location_lat,material,user_id,location_long
0,None,Low,Thriving,1,74.0,Other,False,Medium,LOW,3,...,Russian,2024-02-22,clipped,swarm cells,1,2025-01-29,5.601693,Polystyrene,1,78.517182
1,Pollen patty,Medium,Swarmed,2,84.3,Formic acid,True,Low,LOW,1,...,Russian,2024-02-22,clipped,swarm cells,1,2025-01-29,5.601693,Polystyrene,1,78.517182
2,Sugar syrup,High,Split,3,57.9,Oxalic acid,True,High,LOW,3,...,Russian,2024-02-22,clipped,swarm cells,1,2025-01-29,5.601693,Polystyrene,1,78.517182
3,Pollen patty,High,Split,4,71.8,Thymol,True,Low,HIGH,3,...,Russian,2024-02-22,clipped,swarm cells,1,2025-01-29,5.601693,Polystyrene,1,78.517182
4,Other,High,Swarmed,5,72.6,Thymol,True,Medium,HIGH,2,...,Russian,2024-02-22,clipped,swarm cells,1,2025-01-29,5.601693,Polystyrene,1,78.517182
